In [7]:
import json
import os,sys

In [8]:
label_file_path = '/home/kimseungjun/datasets/My_PT_data/PT_data/train/label.json'
with open(label_file_path,'r') as f:
    labels = json.load(f)

#print(labels)
new_label = {}
for path_name,cls_la in labels.items():
    bn = os.path.basename(path_name)
    new_label[bn]=cls_la


In [ ]:
print(new_label)
new_label_file_path = '/home/kimseungjun/datasets/My_PT_data/PT_data/train/bn_label.json'

with open(new_label_file_path, 'w') as f:
    json.dump(new_label, f, indent=4)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def find_points(obj):
    """
    obj 안에서 point cloud로 보이는 Nx3 배열/텐서를 찾아 반환.
    가장 흔한 키/형태를 우선 탐색.
    """
    # 텐서/넘파이로 바로 들어있는 경우
    if isinstance(obj, torch.Tensor):
        arr = obj.detach().cpu().numpy()
        if arr.ndim == 2 and arr.shape[1] >= 3:
            return arr[:, :3]
    if isinstance(obj, np.ndarray):
        if obj.ndim == 2 and obj.shape[1] >= 3:
            return obj[:, :3]

    # dict 형태
    if isinstance(obj, dict):
        candidate_keys = ["points", "xyz", "pos", "coords", "coord", "pcd", "pointcloud", "verts", "vertices"]
        for k in candidate_keys:
            if k in obj:
                pts = find_points(obj[k])
                if pts is not None:
                    return pts
        # dict 값들 전체를 훑어보기
        for v in obj.values():
            pts = find_points(v)
            if pts is not None:
                return pts

    # list/tuple 형태
    if isinstance(obj, (list, tuple)):
        for v in obj:
            pts = find_points(v)
            if pts is not None:
                return pts

    return None

def vis_pth_pointcloud(pth_path, max_points=200_000, elev=20, azim=45):
    data = torch.load(pth_path, map_location="cpu")
    pts = find_points(data)
    if pts is None:
        raise ValueError("이 .pth에서 Nx3 points를 찾지 못했어. (키 이름/구조가 특이한 듯)")

    # 다운샘플
    n = pts.shape[0]
    if n > max_points:
        idx = np.random.choice(n, max_points, replace=False)
        pts = pts[idx]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], s=0.5)
    ax.view_init(elev=elev, azim=azim)
    ax.set_title(f"{pth_path}  (N={pts.shape[0]})")
    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    plt.show()

# 사용 예시
vis_pth_pointcloud("/home/kimseungjun/datasets/My_PT_data/PT_data/train/0000012632_20250918_152808.pth")